**Tools in Langchain**

Tools are interfaces that an agent, chain, or LLM can use to interact with the world.They combine a few things:

* The name of the tool
* A description of what the tool is
* JSON schema of what the inputs to the tool are
* The function to call
* Whether the result of a tool should be returned directly to the user

It is useful to have all this information because this information can be used to build action-taking systems! The name, description, and JSON schema can be used to prompt the LLM so it knows how to specify what action to take, and then the function to call is equivalent to taking that action.

The simpler the input to a tool is, the easier it is for an LLM to be able to use it. Many agents will only work with tools that have a single string input.

**Search Engine With Tools and Agents**



In [1]:
##Arxiv-Research
# Tools Creation
from langchain.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper



In [2]:
# Used the inbuilt tool of wikipedia
wiki_api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=300)
wiki=WikipediaQueryRun(api_wrapper=wiki_api_wrapper)
wiki.name

'wikipedia'

In [3]:
arxiv_api_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=300)
arxiv=ArxivQueryRun(api_wrapper=arxiv_api_wrapper)
arxiv.name

'arxiv'

In [4]:
tools=[wiki,arxiv]

Custom tools[RAG Tool]

In [31]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [7]:
loader=WebBaseLoader("https://docs.smith.langchain.com")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vector_db=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vector_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F564128E50>, search_kwargs={})

In [9]:
# convert the retriever into tools
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about LangSmith")

retriever_tool.name


'langsmith-search'

In [11]:
tools=[wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\ROHIT GUPTA\\Desktop\\GenAI-Course-Materials-Krish_Naik\\LangChain\\myvenv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=300)),
 Tool(name='langsmith-search', description='Search any information about LangSmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001F559F0C0D0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectors

Run all this tools with Agents and LLM Models

In [12]:
# Tools, LLM-->AgentExecutor
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
import openai
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
openai_api_key=os.getenv("OPENAI_API_KEY")

model=ChatGroq(model_name="Llama3-8b-8192",groq_api_key=groq_api_key)

In [25]:
# prompt template
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt =ChatPromptTemplate.from_messages(
  [
    ("system","""You are a helpful AI assistant.
    If you don't know the answer, just say that you don't know, don't try to make up an answer."""),
     MessagesPlaceholder(variable_name="agent_scratchpad"),
    ("human","{input}")
  ]
)
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Ann

Agents

In [26]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(model,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='

Agent Executer

In [27]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typin

In [28]:
agent_executor.invoke({"input":"Tell me about LangSmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prototyping, to beta testing, to production.

Skip to main conten

{'input': 'Tell me about LangSmith',
 'output': "LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows developers to closely monitor and evaluate their applications, enabling them to ship quickly and with confidence. LangSmith provides observability, evaluation, and prompt engineering tools to help developers build and optimize their LLM applications.\n\nSome of the key features of LangSmith include:\n\n1. Observability: LangSmith provides LLM-native observability, allowing developers to analyze traces, configure metrics, and set up dashboards and alerts.\n2. Evaluation: LangSmith's evaluation tools enable developers to evaluate their application over production traffic, score performance, and receive human feedback on their data.\n3. Prompt Engineering: LangSmith provides tools for iterating on prompts, with automatic version control and collaboration features.\n4. SDK and UI: LangSmith offers a software development kit (SDK) and user

In [29]:
agent_executor.invoke({"input":"What is GenAI?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'GenAI'}`


Page: Generative artificial intelligence
Summary: Generative artificial intelligence (generative AI, GenAI, or GAI) is a subset of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data. These models learn the underlying patterns and structures of their training data and use them to produce new data based on the input, which often comes in the form of natural language prompts.  
Improvements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the 2020s. These include chatbots such as ChatGPT, Copilot, Gemini, and LLaMA; text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video AI generators such as Sora. Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smal

{'input': 'What is GenAI?',
 'output': 'Based on the information provided, GenAI stands for Generative Artificial Intelligence, which is a subset of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.'}

In [30]:
# arxiv 1706.03762
agent_executor.invoke({"input":"What is 1706.03762 research paper about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks The research paper with the ID 1706.03762 is titled "Attention Is All You Need" and it is about the development of a new type of neural network architecture called the Transformer. The paper, published in 2017, introduces a new approach to sequence transduction that is based solely on self-attention mechanisms, and shows that this approach can achieve state-of-the-art results in machine translation tasks.

> Finished chain.


{'input': 'What is 1706.03762 research paper about?',
 'output': 'The research paper with the ID 1706.03762 is titled "Attention Is All You Need" and it is about the development of a new type of neural network architecture called the Transformer. The paper, published in 2017, introduces a new approach to sequence transduction that is based solely on self-attention mechanisms, and shows that this approach can achieve state-of-the-art results in machine translation tasks.'}